In [2]:
import pandas as pd
import os
from Bio import Entrez, SeqIO
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np
import ast

In [ ]:
# Load the data iteratively to avoid crashing the kernel
iterator = pd.read_csv("LLR_final_results.tsv",sep="\t", usecols=['seqName','LLR','privateNucMutations.unlabeledSubstitutions'], dtype={"seqName":str, "LLR": np.float64, 'privateNucMutations.unlabeledSubstitutions':str } , chunksize=1000)
df = pd.concat([chunk for chunk in tqdm.tqdm(iterator, desc='Loading data')])

Loading data: 19114it [01:40, 190.26it/s]


In [ ]:
# Load mutational spectra
spectra = [
    {"name":"BA.1",
     "url": "https://raw.githubusercontent.com/theosanderson/molnupiravir/main/mutational_spectra/BA.1_SBS_spectrum_Ruis.csv"
    },
    {"name":"High G-to-A",
        "url": "https://raw.githubusercontent.com/theosanderson/molnupiravir/main/mutational_spectra/long_phylogenetic_branches/long_branch_spectrum_rescaled.csv"    
    },
]

In [5]:
df['privateNucMutations.unlabeledSubstitutions'].str.split(",")

0           [G204A, C2445T, C4331T, C5621T, C5622T, C6633T...
1                                                         NaN
2                                                         NaN
3                                                   [G19999T]
4                                             [G410T, A7881G]
                                  ...                        
19113491                           [C6629T, A15942G, G19009A]
19113492    [T19053C, T19590C, G24197A, G26104T, C26305A, ...
19113493                              [C44T, G6554A, C18348T]
19113494    [G488A, A2253G, C4940T, C5144T, C21998A, G2626...
19113495                                                  NaN
Name: privateNucMutations.unlabeledSubstitutions, Length: 19113496, dtype: object

In [6]:
def fetch_reference_genome(accession='NC_045512.2'):
    Entrez.email = "theo@theo.io"  
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return str(record.seq)
reference_genome = fetch_reference_genome("NC_045512.2")  # SARS-CoV-2 reference genome

In [7]:
def get_mut_type(mut_string):
    if isinstance(mut_string, str) and len(mut_string) >= 2:
        return mut_string[0] + '>' + mut_string[-1]
    else:
        return ''
df["subs"]=df["privateNucMutations.unlabeledSubstitutions"].apply(lambda x: ','.join([get_mut_type(item) for item in x.split(',')])if isinstance(x, str) else '')
df["subs"].head()

0    G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...
1                                                     
2                                                     
3                                                  G>T
4                                              G>T,A>G
Name: subs, dtype: object

In [8]:
df["Counts"] =df["subs"].apply(lambda x: dict(Counter(x.split(","))) if isinstance(x, str) and x.strip() else {})
df["Counts"].head()

0    {'G>A': 3, 'C>T': 8, 'A>C': 1, 'A>T': 1, 'G>T'...
1                                                   {}
2                                                   {}
3                                           {'G>T': 1}
4                                 {'G>T': 1, 'A>G': 1}
Name: Counts, dtype: object

In [ ]:
# Function to get mutational context of private substitutions from reference genome
def get_context(genome_seq, mutation):
    if isinstance(mutation, str) and len(mutation) >= 2:
        pos = int(mutation[1:-1]) - 1  # -1 as Python uses 0-based indexing
        context = genome_seq[pos-1:pos+2]  # get the base before and after
        return context
    else:
        return ''
df["context"]=df["privateNucMutations.unlabeledSubstitutions"].apply(lambda x: ','.join([get_context(reference_genome, item) for item in x.split(',')])if isinstance(x, str) else '')
df["context"].head() 

0    CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...
1                                                     
2                                                     
3                                                  AGT
4                                              TGG,AAT
Name: context, dtype: object

In [ ]:
# Function to get mutational context including the mutational residue such as 'C[G>A]T'.
def spectrum(subs, contexts):
    if not isinstance(subs, str) or not isinstance(contexts, str):
        return ''
    if not subs.strip() or not contexts.strip():
        return ''
    subs= subs.split(',')
    contexts = contexts.split(',')
    spectra = []
    for mutation, context in zip(subs, contexts): 
        if len(context) >= 2 and mutation:
            spectra.append(f"{context[0]}[{mutation}]{context[-1]}")
        else:
            continue
    return ','.join(spectra)
   
df["spectrum"] = df.apply(lambda row: spectrum(row["subs"], row["context"]), axis=1)
df["spectrum"].head()

0    C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...
1                                                     
2                                                     
3                                              A[G>T]T
4                                      T[G>T]G,A[A>G]T
Name: spectrum, dtype: object

In [ ]:
# Function to count G>A contexts. 
def count_GtoA(spectrum):
    counts = Counter()
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'G>A':
            counts[mut]+=1
    return counts

df["context_counts"] = df["spectrum"].apply(count_GtoA)
df["context_counts"].head()

            

0    {'C[G>A]T': 1, 'A[G>A]G': 1, 'T[G>A]C': 1}
1                                            {}
2                                            {}
3                                            {}
4                                            {}
Name: context_counts, dtype: object

In [ ]:
# All 16 possible G to A substitutions.
bases = ["A","C","G","T"]
possible_contexts = []
for a in bases:
    for b in bases:
        possible_contexts.append(f"{a}[G>A]{b}")
possible_contexts

['A[G>A]A',
 'A[G>A]C',
 'A[G>A]G',
 'A[G>A]T',
 'C[G>A]A',
 'C[G>A]C',
 'C[G>A]G',
 'C[G>A]T',
 'G[G>A]A',
 'G[G>A]C',
 'G[G>A]G',
 'G[G>A]T',
 'T[G>A]A',
 'T[G>A]C',
 'T[G>A]G',
 'T[G>A]T']

In [ ]:
# Function to get the proportions of each G to A mutational context per row.
def get_proportion(df):
    dict = {}
    total = sum(df.values())
    for key, value in df.items():
        dict[key] = value/total
       
    return dict

df["proportions"] = df.apply(lambda row: get_proportion(row["context_counts"]), axis=1)
df["proportions"].head(20)

0     {'C[G>A]T': 0.3333333333333333, 'A[G>A]G': 0.3...
1                                                    {}
2                                                    {}
3                                                    {}
4                                                    {}
5                                                    {}
6                                      {'A[G>A]C': 1.0}
7                                                    {}
8                                                    {}
9                                                    {}
10                                                   {}
11                                     {'C[G>A]T': 1.0}
12                                     {'A[G>A]G': 1.0}
13                                                   {}
14                                                   {}
15                                                   {}
16                                                   {}
17                                     {'C[G>A]G

In [ ]:
# Splitting the data to include only sequences with Log likelihood Ratios > 6
df_high_llrs = df[df["LLR"]>6]
df_high_llrs.tail()

,seqName,privateNucMutations.unlabeledSubstitutions,LLR,subs,Counts,context,spectrum,context_counts,proportions
19072444,hCoV-19/New Zealand/23XA0013/2022|2022-12-30|2...,"C906T,T926C,A1818G,G2030A,A3375G,G3782A,G4712A...",6.916040,"C>T,T>C,A>G,G>A,A>G,G>A,G>A,T>C,C>T,G>A,T>C,G>...","{'C>T': 11, 'T>C': 3, 'A>G': 4, 'G>A': 9}","ACT,CTT,AAA,TGA,AAT,TGT,TGT,CTT,GCT,GGG,CTT,TG...","A[C>T]T,C[T>C]T,A[A>G]A,T[G>A]A,A[A>G]T,T[G>A]...","{'T[G>A]A': 3, 'T[G>A]T': 2, 'G[G>A]G': 1, 'T[...","{'T[G>A]A': 0.3333333333333333, 'T[G>A]T': 0.2..."
19072863,hCoV-19/Puerto Rico/PR-CDC-ASC210064784/2021|2...,"C1437T,G3446A,C4891T,C9430T,C9714T,C9745T,C103...",8.784146,"C>T,G>A,C>T,C>T,C>T,C>T,C>T,C>T,A>G,C>T,C>T,C>...","{'C>T': 20, 'G>A': 2, 'A>G': 2, 'T>C': 1}","TCT,AGC,CCA,TCG,ACA,ACC,GCC,ACT,AAA,ACA,ACT,TC...","T[C>T]T,A[G>A]C,C[C>T]A,T[C>T]G,A[C>T]A,A[C>T]...","{'A[G>A]C': 1, 'G[G>A]T': 1}","{'A[G>A]C': 0.5, 'G[G>A]T': 0.5}"
19081625,hCoV-19/South Korea/KDCA214548/2023|2023-02-03...,"G862A,G1358A,G1709A,C1812T,C2638T,A3181G,G3335...",6.305596,"G>A,G>A,G>A,C>T,C>T,A>G,G>A,A>G,T>C,C>T,T>C,A>...","{'G>A': 12, 'C>T': 11, 'A>G': 3, 'T>C': 2, 'T>...","AGT,TGT,TGC,GCT,TCG,AAG,TGA,AAG,TTA,ACC,TTA,CA...","A[G>A]T,T[G>A]T,T[G>A]C,G[C>T]T,T[C>T]G,A[A>G]...","{'A[G>A]T': 2, 'T[G>A]T': 1, 'T[G>A]C': 5, 'T[...","{'A[G>A]T': 0.16666666666666666, 'T[G>A]T': 0...."
19083442,hCoV-19/South Korea/KDCA214804/2023|2023-02-04...,"C44T,G1085A,G1111A,G4488A,C5842T,A11386G,C1145...",7.432855,"C>T,G>A,G>A,G>A,C>T,A>G,C>T,T>C,G>A,C>T,G>A,G>...","{'C>T': 9, 'G>A': 12, 'A>G': 1, 'T>C': 2, 'G>C...","TCG,TGT,AGA,GGT,ACA,GAG,GCC,CTA,TGC,GCG,TGC,AG...","T[C>T]G,T[G>A]T,A[G>A]A,G[G>A]T,A[C>T]A,G[A>G]...","{'T[G>A]T': 5, 'A[G>A]A': 1, 'G[G>A]T': 1, 'T[...","{'T[G>A]T': 0.4166666666666667, 'A[G>A]A': 0.0..."
19101133,hCoV-19/Hong Kong/HK-HKPU-PU24MB306462/2024|20...,"C186T,G521A,C1063T,G1729A,C2137T,G2284A,C2724T...",6.507057,"C>T,G>A,C>T,G>A,C>T,G>A,C>T,A>G,G>A,C>T,A>G,C>...","{'C>T': 13, 'G>A': 5, 'A>G': 3, 'T>C': 2}","GCA,GGT,TCA,TGG,TCA,AGA,CCA,AAA,GGT,ACC,CAT,AC...","G[C>T]A,G[G>A]T,T[C>T]A,T[G>A]G,T[C>T]A,A[G>A]...","{'G[G>A]T': 2, 'T[G>A]G': 1, 'A[G>A]A': 1, 'G[...","{'G[G>A]T': 0.4, 'T[G>A]G': 0.2, 'A[G>A]A': 0...."


In [ ]:
# Flattening the proportions of GtoA mutations into a table to calculate their mean values.
proportions_list = pd.json_normalize(df_high_llrs['proportions'])
proportions_list.index = df_high_llrs.index 
proportions_list.head()

,T[G>A]C,T[G>A]T,C[G>A]T,G[G>A]A,A[G>A]G,T[G>A]G,G[G>A]T,T[G>A]A,C[G>A]C,A[G>A]C,C[G>A]G,G[G>A]C,A[G>A]A,A[G>A]T,G[G>A]G,C[G>A]A
16264,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31127,0.100000,0.300000,0.05,0.05,0.1,0.100000,0.100000,0.150000,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,NaN,0.428571,NaN,NaN,NaN,NaN,0.142857,0.285714,NaN,0.142857,NaN,NaN,NaN,NaN,NaN,NaN
38309,0.230769,0.153846,NaN,NaN,NaN,0.230769,0.076923,0.153846,NaN,NaN,0.076923,0.076923,NaN,NaN,NaN,NaN
41700,0.100000,0.100000,0.10,NaN,NaN,0.200000,NaN,0.200000,0.10,0.100000,NaN,NaN,0.1,NaN,NaN,NaN


In [ ]:
# Create a new data frame "df_Mov" that includes seqNames,proportions and LLRs of Molnupiravir likely sequences,
df_Mov = pd.concat([df_high_llrs[["seqName"]],proportions_list,df_high_llrs[["LLR"]]], axis=1)
df_Mov.tail()

,seqName,T[G>A]C,T[G>A]T,C[G>A]T,G[G>A]A,A[G>A]G,T[G>A]G,G[G>A]T,T[G>A]A,C[G>A]C,A[G>A]C,C[G>A]G,G[G>A]C,A[G>A]A,A[G>A]T,G[G>A]G,C[G>A]A,LLR
19031669,hCoV-19/South Korea/KDCA253906/2023|2023-07-07...,0.250000,0.125000,NaN,NaN,NaN,0.125000,0.125000,0.250000,NaN,0.125000,NaN,NaN,NaN,NaN,NaN,NaN,6.024234
19035535,hCoV-19/Trinidad and Tobago/122171/2022|2022-0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.333333,NaN,NaN,0.333333,NaN,NaN,NaN,6.103049
19039378,hCoV-19/New Zealand/22XA3574/2022|2022-10-05|2...,0.100000,0.400000,NaN,NaN,NaN,0.200000,NaN,0.200000,NaN,NaN,NaN,NaN,NaN,0.100000,NaN,NaN,9.475768
19057905,hCoV-19/South Korea/KDCA277654/2023|2023-11-21...,0.090909,0.181818,NaN,NaN,0.090909,NaN,NaN,NaN,0.090909,0.090909,NaN,0.090909,NaN,0.363636,NaN,NaN,6.225571
19066808,hCoV-19/South Korea/KDCA216693/2023|2023-02-12...,0.333333,0.222222,0.111111,NaN,NaN,0.111111,NaN,0.111111,NaN,NaN,NaN,NaN,NaN,0.111111,NaN,NaN,9.041364
19072444,hCoV-19/New Zealand/23XA0013/2022|2022-12-30|2...,0.222222,0.222222,0.111111,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0.111111,NaN,6.916040
19072863,hCoV-19/Puerto Rico/PR-CDC-ASC210064784/2021|2...,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,NaN,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,8.784146
19081625,hCoV-19/South Korea/KDCA214548/2023|2023-02-03...,0.416667,0.083333,NaN,0.083333,NaN,NaN,NaN,0.166667,NaN,0.083333,NaN,NaN,NaN,0.166667,NaN,NaN,6.305596
19083442,hCoV-19/South Korea/KDCA214804/2023|2023-02-04...,0.250000,0.416667,NaN,NaN,NaN,NaN,0.083333,0.083333,NaN,NaN,NaN,NaN,0.083333,0.083333,NaN,NaN,7.432855
19101133,hCoV-19/Hong Kong/HK-HKPU-PU24MB306462/2024|20...,NaN,NaN,NaN,0.200000,NaN,0.200000,0.400000,NaN,NaN,NaN,NaN,NaN,0.200000,NaN,NaN,NaN,6.507057


In [ ]:
# calculate mean values for each G to A context and append the mean row to df_Mov.
context_cols = [col for col in df_Mov.columns if col not in ["seqName", "LLR"]]
mean_row = df_Mov[context_cols].fillna(0).mean()
mean_row['seqName'] = 'Mean'
#print(mean_row)

df_Mov = pd.concat([df_Mov, pd.DataFrame([mean_row])], ignore_index=True)
print(df_Mov.tail())





                                               seqName   T[G>A]C   T[G>A]T  \
861  hCoV-19/Puerto Rico/PR-CDC-ASC210064784/2021|2...       NaN       NaN   
862  hCoV-19/South Korea/KDCA214548/2023|2023-02-03...  0.416667  0.083333   
863  hCoV-19/South Korea/KDCA214804/2023|2023-02-04...  0.250000  0.416667   
864  hCoV-19/Hong Kong/HK-HKPU-PU24MB306462/2024|20...       NaN       NaN   
865                                               Mean  0.160377  0.261640   

      C[G>A]T   G[G>A]A   A[G>A]G   T[G>A]G   G[G>A]T   T[G>A]A   C[G>A]C  \
861       NaN       NaN       NaN       NaN  0.500000       NaN       NaN   
862       NaN  0.083333       NaN       NaN       NaN  0.166667       NaN   
863       NaN       NaN       NaN       NaN  0.083333  0.083333       NaN   
864       NaN  0.200000       NaN  0.200000  0.400000       NaN       NaN   
865  0.054176  0.019890  0.025408  0.064058  0.038508  0.100146  0.019887   

      A[G>A]C   C[G>A]G   G[G>A]C   A[G>A]A   A[G>A]T   G[G>A]G   C[

In [ ]:
# Save
df_Mov.to_csv("Mov_means.tsv",sep = "\t")

In [ ]:
# Repeat the same process for LLRs < 6 (Likely Non-Mov seqs)
df_low_llrs = df[df["LLR"]<6]
df_low_llrs.head()

,seqName,privateNucMutations.unlabeledSubstitutions,LLR,subs,Counts,context,spectrum,context_counts,proportions
19113491,hCoV-19/Northern Ireland/NIRE-00422b/2021|2021...,"C6629T,A15942G,G19009A",0.730905,"C>T,A>G,G>A","{'C>T': 1, 'A>G': 1, 'G>A': 1}","CCT,CAT,AGA","C[C>T]T,C[A>G]T,A[G>A]A",{'A[G>A]A': 1},{'A[G>A]A': 1.0}
19113492,hCoV-19/South Africa/NICD-N21220/2021|2021-09-...,"T19053C,T19590C,G24197A,G26104T,C26305A,T26446...",-2.779755,"T>C,T>C,G>A,G>T,C>A,T>C,A>G,C>T,C>T,C>T","{'T>C': 3, 'G>A': 1, 'G>T': 1, 'C>A': 1, 'A>G'...","TTA,ATA,AGC,TGA,TCT,TTC,TAA,ACA,ACT,TCC","T[T>C]A,A[T>C]A,A[G>A]C,T[G>T]A,T[C>A]T,T[T>C]...",{'A[G>A]C': 1},{'A[G>A]C': 1.0}
19113493,hCoV-19/South Korea/KDCA58664/2022|2022-05-04|...,"C44T,G6554A,C18348T",1.377532,"C>T,G>A,C>T","{'C>T': 2, 'G>A': 1}","TCG,GGC,CCA","T[C>T]G,G[G>A]C,C[C>T]A",{'G[G>A]C': 1},{'G[G>A]C': 1.0}
19113494,hCoV-19/South Korea/KDCA259323/2023|2023-08-05...,"G488A,A2253G,C4940T,C5144T,C21998A,G26262A,C26...",1.258031,"G>A,A>G,C>T,C>T,C>A,G>A,C>T,C>T","{'G>A': 2, 'A>G': 1, 'C>T': 4, 'C>A': 1}","GGA,CAA,ACT,TCT,CCA,CGG,TCT,ACA","G[G>A]A,C[A>G]A,A[C>T]T,T[C>T]T,C[C>A]A,C[G>A]...","{'G[G>A]A': 1, 'C[G>A]G': 1}","{'G[G>A]A': 0.5, 'C[G>A]G': 0.5}"
19113495,hCoV-19/New Zealand/22ZA4228/2022|2022-11-14|2...,NaN,0.000000,,{},,,{},{}


In [16]:
proportions_list_low = pd.json_normalize(df_low_llrs['proportions'])
proportions_list_low.index = df_low_llrs.index 
proportions_list_low.head()

,C[G>A]T,A[G>A]G,T[G>A]C,A[G>A]C,C[G>A]G,T[G>A]G,G[G>A]T,A[G>A]A,T[G>A]T,A[G>A]T,G[G>A]C,G[G>A]G,T[G>A]A,C[G>A]C,G[G>A]A,C[G>A]A
0,0.333333,0.333333,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create a new dataframe "df_Normal" for low LLR values
df_Normal = pd.concat([df_low_llrs[["seqName"]],proportions_list_low,df_low_llrs[["LLR"]]], axis=1)
df_Normal.head()

,seqName,C[G>A]T,A[G>A]G,T[G>A]C,A[G>A]C,C[G>A]G,T[G>A]G,G[G>A]T,A[G>A]A,T[G>A]T,A[G>A]T,G[G>A]C,G[G>A]G,T[G>A]A,C[G>A]C,G[G>A]A,C[G>A]A,LLR
19113486,hCoV-19/Northern Ireland/NIRE-00efe9/2021|2021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.661912
19113487,hCoV-19/South Korea/KDCA15722s/2022|2022-02-06...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
19113488,hCoV-19/South Korea/KDCA124774/2022|2022-09-07...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.412451
19113489,hCoV-19/Puerto Rico/PR-CVL-032418/2025|2025-04...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.210447
19113490,hCoV-19/South Korea/KDCA189331/2022|2022-12-22...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.599670
19113491,hCoV-19/Northern Ireland/NIRE-00422b/2021|2021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.730905
19113492,hCoV-19/South Africa/NICD-N21220/2021|2021-09-...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.779755
19113493,hCoV-19/South Korea/KDCA58664/2022|2022-05-04|...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.377532
19113494,hCoV-19/South Korea/KDCA259323/2023|2023-08-05...,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,1.258031
19113495,hCoV-19/New Zealand/22ZA4228/2022|2022-11-14|2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [ ]:
# Used gc.collect to free up some memory as df_Normal is huge.
# Calculate mean values per G to A context, append to the end of the df and save.

context_cols = [col for col in df_Normal.columns if col not in ["seqName", "LLR"]]
df_Normal[context_cols].dtypes
mean_row = df_Normal[context_cols].fillna(0).mean()
mean_row['seqName'] = 'Mean'
print(mean_row)




C[G>A]T    0.020609
A[G>A]G    0.032408
T[G>A]C    0.015557
A[G>A]C    0.018432
C[G>A]G    0.012249
T[G>A]G    0.023983
G[G>A]T    0.011146
A[G>A]A    0.026299
T[G>A]T    0.040269
A[G>A]T    0.027112
G[G>A]C    0.012488
G[G>A]G    0.008023
T[G>A]A    0.018122
C[G>A]C    0.015707
G[G>A]A    0.011489
C[G>A]A    0.006896
seqName        Mean
dtype: object


In [21]:
df_Normal = pd.concat([df_Normal, pd.DataFrame([mean_row])], ignore_index=True)
print(df_Normal.head())


                                             seqName   C[G>A]T   A[G>A]G  \
0  hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...  0.333333  0.333333   
1  hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...       NaN       NaN   
2  hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...       NaN       NaN   
3  hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...       NaN       NaN   
4  hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...       NaN       NaN   

    T[G>A]C  A[G>A]C  C[G>A]G  T[G>A]G  G[G>A]T  A[G>A]A  T[G>A]T  A[G>A]T  \
0  0.333333      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

   G[G>A]C  G[G>A]G  T[G>A]A  C[G>A]C  G[G>A]A  C[G>A]A       LLR  
0     

In [22]:

df_Normal.to_csv("Normal_means.tsv",sep = "\t")